In [1]:
import nltk
from nltk.stem import LancasterStemmer # For stemming words
stemmer = LancasterStemmer()

import numpy as np
import tflearn as tfl
import tensorflow as tf
import random
import json
import pickle

with open('intents.json','r') as f:
    data = json.load(f)

words=[]
labels=[]
doc_x=[]
doc_y=[]
    
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        doc_x.append(wrds)
        doc_y.append(intent['tag'])
            
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
            
words = [stemmer.stem(w.lower()) for w in words if w != '?']
words = sorted(list(set(words)))
    
labels = sorted(labels)
    
training = []
output = []
    
out_empty = [0 for _ in range(len(labels))]
    
for x, doc in enumerate(doc_x):
    bag = []
        
wrds = [stemmer.stem(w) for w in doc]
        
for w in words:
    if w in wrds:
        bag.append(1)
    else:
        bag.append(0)
                
output_row = out_empty[:]
output_row[labels.index(doc_y[x])] = 1
        
training.append(bag)
output.append(output_row)
        
training = np.array(training)
output = np.array(output)

net = tfl.input_data(shape=[None, len(training[0])])
net = tfl.fully_connected(net, 8)
net = tfl.fully_connected(net, 8)
net = tfl.fully_connected(net, len(output[0]), activation='softmax')
net = tfl.regression(net)

model = tfl.DNN(net)


model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return np.array(bag)

def chat():
    print('Start chatting with the bot!')
    while True:
        inp = input('You:')
        if inp.lower() == 'quit':
            break
        
        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
        
        print(random.choice(responses))
        
chat()

C:\Users\Sebin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)



ImportError: cannot import name 'is_sequence' from 'tensorflow.python.util.nest' (C:\Users\Sebin\anaconda3\lib\site-packages\tensorflow\python\util\nest.py)